# Functions for retrieving data, making basic calculations, and running Monte Carlo simulations

## See [Basic usage](#Basic-usage) below if you just want to see how to use the module and don't care to read the code.

### Imports

In [32]:
import pandas as pd
from pathlib import Path
import numpy as np
import datetime as dt
from pathlib import Path
import os
from datetime import datetime, timedelta

In [33]:
from sqlalchemy import create_engine

In [34]:
np.random.seed(42)

### Connect to DB

In [35]:
# connectionString = "dbname='project01' user='pgadmin@fintech-postgres' host='fintech-postgres.postgres.database.azure.com' password='CgYRz!)[4pfB' port='5432' sslmode='true'"
engine = create_engine(
    "postgresql://pgadmin@fintech-postgres:CgYRz!)[4pfB@fintech-postgres.postgres.database.azure.com:5432/project01")

### Functions definitions

In [36]:
def get_portfolio_daily_returns(portfolio_name, *, covid_phase):
    '''
    portfolio_name := string,
    covid_phase := boolean (whether to include data including dates 2/19/20 - 4/29/20)
    
    Query the postgres db.
    Run basic calculations.
    
    Return DF of portfolio daily returns that will be used in the rest of the analysis.
    '''
    
    q = f'''
        select portfolio_name, weight, "date" as close_date, 
            close_price, ticker_name, covidineffect 
        from portfolio p
            inner join portfolio_tickers pt
            on p.portfolio_id = pt.portfolio_id
            inner join price
            on pt.ticker_id = price.ticker_id
        where price.covidineffect = {covid_phase} and p.portfolio_name = '{portfolio_name}'
        '''
    portfolio_df = pd.read_sql(q, engine)
    
    # Pivot the table so dates are index
    portfolio_df = portfolio_df.pivot(index='close_date', columns='ticker_name', values='close_price')
    
    daily_returns = portfolio_df.pct_change()
    
    daily_returns = daily_returns.dropna()
    
    return daily_returns

In [37]:
def portfolio_figures(pf_daily_returns):
    '''
    Returns tuple(last_close_price, returns_mean, returns_std)
    which can be passed in directly to `run_simulations`.
    '''
    return pf_daily_returns.iloc[-1], pf_daily_returns.mean(), pf_daily_returns.std()

In [38]:
def run_simulations(pf_figures, num_records, num_simulations):
    '''
    pf_data := tuple(last_close_prices, returns_means, returns_stds)
    
    Runs `num_simulations` Monte Carlo simulations each of `num_records`.
    
    Returns DF of cumulative returns
    '''
    
    monte_carlo_cum_returns_df = pd.DataFrame()
    
    last_close_prices, returns_means, returns_stds = pf_figures
    
    for m in range(num_simulations):
        simulated_prices_all = [[price] for price in last_close_prices]
        
        for _ in range(num_records):
            
            for i, prices in enumerate(simulated_prices_all):
                sim_price = prices[-1] * (1 + np.random.normal(returns_means[i], returns_stds[i]))
                prices.append(sim_price)

        monte_carlo_price_df = pd.DataFrame(dict(zip(last_close_prices.index, simulated_prices_all)))

        sim_daily_returns = monte_carlo_price_df.pct_change()

        weighted_returns = sim_daily_returns.dot([0.25] * 4)

        monte_carlo_cum_returns_df[m] = (1 + weighted_returns.fillna(0)).cumprod()
    
    return monte_carlo_cum_returns_df

## Basic usage

The main functionality here is wrapped in 3 functions: `get_portfolio_daily_returns`, `portfolio_figures`, and `run_simulations`. You can run `portfolio_figures?` to view the doc string or just scroll up and look over the function definition.

### Examples

Suppose we want to compare the cumulative returns of the 5 portfolios using just the data before COVID.
Here's how we could get those data:

In [39]:
# List of portfolios
portfolios = ['pharma', 'food', 'communication', 'hedge', 'financial']

# Dictionary for storing the results of the simulations
sim_results = dict.fromkeys(portfolios)

# Do the simulations and store results
for pf in portfolios:
    # covid=False here since we don't want data after 2/19/2020
    # Also notice the keyword `covid` is required here
    pf_returns = get_portfolio_daily_returns(pf, covid_phase=False)
    pf_figures = portfolio_figures(pf_returns)
    pf_sims = run_simulations(pf_figures, 105, 100)
    sim_results[pf] = pf_sims

In [40]:
# Results of one of the portfolios
pharma_sim_results = sim_results['pharma']
pharma_sim_results.tail()

,0
26,0.969090
27,0.975597
28,0.970836
29,0.977458
30,0.980093


If we just want to get the (actual) daily returns of a portfolio, we can do that easily.

In [41]:
pharma_pf_real_daily_returns = get_portfolio_daily_returns('pharma', covid_phase=False)
pharma_pf_real_daily_returns.head()

ticker_name,ABT,JNJ,MRK,PFE
close_date,,,,
2019-02-20,0.007200,0.004864,0.002398,-0.009390
2019-02-21,-0.005295,-0.006821,0.005036,-0.001422
2019-02-22,0.013175,0.008714,0.011775,0.019459
2019-02-25,0.009589,-0.002928,-0.004829,0.002793
2019-02-26,-0.001431,-0.000661,0.004479,-0.001393


Find the covariance of a stock:

In [43]:
# JNJ daily returns
pharma = get_portfolio_daily_returns['pharma']

# S&P 500 real returns
sp500_real_daily_returns = get_portfolio_daily_returns('S&P500', covid_phase=False)

# JNJ covariance
pharma_cov_non_covid = jnj.cov(sp500_real_daily_returns['SP500'])
pharma.cov(sp500_real_daily_returns['SP500'])
pharma_cov_non_covid

TypeError: 'function' object is not subscriptable

In [ ]:
# JNJ daily returns


# S&P 500 real returns


# JNJ covariance


## Tests

In [ ]:
# Tests here


In [ ]:
cumulative_returns = [sim_results[pf].iloc[-1] for pf in portfolios]
